# Gamut Mapping - Medicina 01

In [1]:
%matplotlib widget

In [2]:
from __future__ import division, unicode_literals

import colour
import ipympl.backend_nbagg
import ipywidgets
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.interpolate
from matplotlib.collections import PathCollection
from matplotlib._layoutbox import plot_children

colour.plotting.colour_style()

plt.style.use('dark_background')

colour.utilities.describe_environment();

colour.utilities.filter_warnings(*[True] * 4)

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.7.6 (default, Dec 30 2019, 19:38:26)                       *
*                [Clang 11.0.0 (clang-1100.0.33.16)]                          *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : v0.3.15-126-gd9ef0a5a                                        *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.8.0                                                       *
*       matplotlib : 3.0.3                                                    *
*       numpy : 1.18.4                                                        *
*       scipy : 1.4.1                   

## Colour Wheel Generation

In [3]:
def colour_wheel(samples=1024, clip_circle=True, method='Colour'):
    xx, yy = np.meshgrid(
        np.linspace(-1, 1, samples), np.linspace(-1, 1, samples))

    S = np.sqrt(xx ** 2 + yy ** 2)    
    H = (np.arctan2(xx, yy) + np.pi) / (np.pi * 2)

    HSV = colour.utilities.tstack([H, S, np.ones(H.shape)])
    RGB = colour.HSV_to_RGB(HSV)

    if clip_circle == True:
        RGB[S > 1] = 0
        A = np.where(S > 1, 0, 1)
    else:
        A = np.ones(S.shape)

    if method.lower()== 'matplotlib':
        RGB = colour.utilities.orient(RGB, '90 CW')
    elif method.lower()== 'nuke':
        RGB = colour.utilities.orient(RGB, 'Flip')
        RGB = colour.utilities.orient(RGB, '90 CW')

    R, G, B = colour.utilities.tsplit(RGB)
    
    return colour.utilities.tstack([R, G, B, A])

## Utility Functions & Objects

In [4]:
DEFAULT_BOX_DECORATION_WIDTH = 4
MPL_BOX_DECORATION_WIDTH = 28


def batch(sequence, count):
    for i in range(0, len(sequence), count):
        yield sequence[i:i + count]
        

def border_layout():
    return ipywidgets.widgets.Layout(
        border='solid {0}px #222'.format(4),
        margin='{0}px'.format(DEFAULT_BOX_DECORATION_WIDTH),
        padding='0')

## Gamut Medicina Base

In [5]:
class GamutMedicinaBaseWidget(ipywidgets.widgets.Box):
    def __init__(self,
                 reference_image_path=None,
                 working_space=colour.models.ACES_CG_COLOURSPACE,
                 reference_space=colour.models.ACES_2065_1_COLOURSPACE,
                 display_space=colour.models.sRGB_COLOURSPACE,
                 colour_wheel_samples=256,
                 image_decimation=20,
                 figure_size=(11, None),
                 image_ratio=3,
                 padding=0.005,
                 spacing=0.005,
                 show_labels=False,
                 enable_colour_wheel=True,
                 enable_chromaticity_diagram=True,
                 debug_layout=False):
        super().__init__()

        self._reference_image_path = reference_image_path
        self._working_space = working_space
        self._reference_space = reference_space
        self._display_space = display_space
        self._colour_wheel_samples = colour_wheel_samples
        self._image_decimation = image_decimation
        self._figure_size = figure_size
        self._image_ratio = image_ratio
        self._padding = padding
        self._spacing = spacing
        self._show_labels = show_labels
        self._enable_colour_wheel = enable_colour_wheel
        self._enable_chromaticity_diagram = enable_chromaticity_diagram

        self._enable = True

        self._reference_image = None
        self._reference_image_working = None
        self._reference_image_mapped = None
        self._reference_image_display = None

        if self._enable_colour_wheel:
            self._colour_wheel_RGBA = None
            self._A_w = None
            self._colour_wheel = None

        if self._enable_chromaticity_diagram:
            self._decimated_image_working = None
            self._decimated_image_display = None

        self._colour_checker = None
        self._colour_checker_uv = None

        self._output = None
        self._figure = None
        self._grid_spec = None
        self._reference_image_axes = None
        self._reference_image_imshow = None

        if self._enable_colour_wheel:
            self._colour_wheel_cartersian_axes = None
            self._colour_wheel_imshow = None
            self._colour_wheel_polar_axes = None

        if self._enable_chromaticity_diagram:
            self._chromaticity_diagram_axes = None
            self._scatter_offsets_i = None
            self._scatter_facecolor_i = None

        self._disable_medidicina_CheckBox = None
        self._global_controls_HBox = None
        self._controls_Tab = None

        self.initialize_data()
        self.initialize_axes()
        self.initialise_widgets()
        self.attach_callbacks()

        if debug_layout:
            plot_children(self._figure, self._figure._layoutbox, printit=False)

    def initialize_data(self):
        # *** Reference Image ***
        self._reference_image = (colour.read_image(self._reference_image_path)
                                 if self._reference_image_path is not None else
                                 np.random.rand(540, 960, 3))
        self._reference_image_path = (self._reference_image_path
                                      if self._reference_image_path is not None
                                      else 'Random.exr')
        self._reference_image_working = self.reference_space_to_working_space(
            self._reference_image)
        self._reference_image_mapped = self._reference_image
        self._reference_image_display = self.working_space_to_display_space(
            self._reference_image_working)

        # *** Colour Wheel ***
        if self._enable_colour_wheel:
            self._colour_wheel_RGBA = colour_wheel(
                self._colour_wheel_samples,
                clip_circle=True,
                method='Matplotlib')
            self._A_w = self._colour_wheel_RGBA[:, :, -1]
            self._colour_wheel = self._colour_wheel_RGBA[:, :, 0:3]

        # *** Decimated Image, i.e. Scatter ***
        if self._enable_chromaticity_diagram:
            self._decimated_image_working = (
                self._reference_image_working[::self._image_decimation, ::
                                              self._image_decimation, :])
            self._decimated_image_display = (
                self.working_space_to_display_space(
                    self._decimated_image_working))

        # *** Colour Checker
        colour_checker = colour.COLOURCHECKERS[
            'ColorChecker24 - After November 2014']
        colour_checker_data = colour.utilities.as_float_array(
            list(colour_checker.data.values()))
        self._colour_checker_uv = colour.xy_to_Luv_uv(
            colour_checker_data[:, 0:2])

    def initialize_axes(self):
        self._output = ipywidgets.widgets.Output()

        image_height, image_width, channels = self._reference_image.shape
        if self._figure_size[-1] is None:
            if self._enable_colour_wheel or self._enable_chromaticity_diagram:
                width = image_width + image_width / self._image_ratio
            else:
                width = image_width
            ratio = self._figure_size[0] * 100 / width
            figure_size = (width / 100 * ratio, image_height / 100 * ratio)
        else:
            figure_size = self._figure_size

        with self._output:
            self._figure = plt.figure(
                figsize=figure_size,
                constrained_layout=True,
                facecolor='black')
            self._figure.canvas.toolbar_visible = False
            self._figure.canvas.header_visible = False
            self._figure.canvas.footer_visible = True
            self._figure.canvas.resizable = False

        if self._enable_colour_wheel and self._enable_chromaticity_diagram:
            rows = columns = 2
            colour_wheel_indices = 0, 0
            chromaticity_diagram_indices = 1, 0
            reference_image_indices = slice(0, None, None), slice(
                1, None, None)
            width_ratios = [1, self._image_ratio]
        elif not self._enable_colour_wheel and not self._enable_chromaticity_diagram:
            rows = columns = 1
            colour_wheel_indices = chromaticity_diagram_indices = None
            reference_image_indices = 0, 0
            width_ratios = None
        else:
            rows = 1
            columns = 2
            colour_wheel_indices = chromaticity_diagram_indices = 0, 0
            reference_image_indices = 0, 1
            width_ratios = [1, self._image_ratio]

        self._grid_spec = gridspec.GridSpec(
            ncols=columns,
            nrows=rows,
            figure=self._figure,
            width_ratios=width_ratios,
            wspace=self._spacing,
            hspace=self._spacing)
        self._figure.set_constrained_layout_pads(
            w_pad=self._padding,
            h_pad=self._padding,
            hspace=self._spacing,
            wspace=self._spacing)

        # Colour Wheel Axes
        if self._enable_colour_wheel:
            self._colour_wheel_cartersian_axes = self._figure.add_subplot(
                self._grid_spec[colour_wheel_indices[0], colour_wheel_indices[
                    1]],
                label='Cartesian Axes')
            self._colour_wheel_cartersian_axes.axis('off')
            self._colour_wheel_imshow = self._colour_wheel_cartersian_axes.imshow(
                np.clip(self._colour_wheel_RGBA, 0, 1), extent=[0, 1, 0, 1])

            self._colour_wheel_polar_axes = self._figure.add_subplot(
                self._grid_spec[colour_wheel_indices[0], colour_wheel_indices[
                    1]],
                projection='polar',
                label='Polar Axes')
            self._colour_wheel_polar_axes.set_xlim(0, np.pi * 2)
            self._colour_wheel_polar_axes.set_ylim(0, 1)
            self._colour_wheel_polar_axes.patch.set_alpha(0)
            self._colour_wheel_polar_axes.grid(alpha=0.15)
            if not self._show_labels:
                self._colour_wheel_polar_axes.set_xticklabels([])
            self._colour_wheel_polar_axes.set_yticklabels([])

        # Chromaticity Diagram Axes
        if self._enable_chromaticity_diagram:
            self._chromaticity_diagram_axes = self._figure.add_subplot(
                self._grid_spec[chromaticity_diagram_indices[0],
                                chromaticity_diagram_indices[1]])
            self._chromaticity_diagram_axes.patch.set_alpha(0)
            (colour.plotting.
             plot_RGB_chromaticities_in_chromaticity_diagram_CIE1976UCS(
                 self._decimated_image_working,
                 colourspace=self._working_space,
                 colourspaces=['sRGB'],
                 axes=self._chromaticity_diagram_axes,
                 standalone=False,
                 spectral_locus_colours='RGB',
                 spectral_locus_labels=[],
                 diagram_opacity=0,
                 scatter_kwargs={'s': 1},
                 title=str(),
                 x_label=str(),
                 y_label=str(),
                 legend=False))
            self._chromaticity_diagram_axes.tick_params(
                axis='y', which='both', direction='in')
            self._chromaticity_diagram_axes.tick_params(
                axis='x', which='both', direction='in')
            self._chromaticity_diagram_axes.minorticks_off()
            self._chromaticity_diagram_axes.grid(alpha=0.15)
            for collection in self._chromaticity_diagram_axes.collections:
                if isinstance(collection, PathCollection):
                    self._scatter_path_collection = collection
                    break
            if not self._show_labels:
                self._chromaticity_diagram_axes.set_xticklabels([])
                self._chromaticity_diagram_axes.set_yticklabels([])

            self._scatter_offsets_i = self._scatter_path_collection.get_offsets(
            )
            self._scatter_facecolor_i = self._scatter_path_collection.get_facecolor(
            )
            self._chromaticity_diagram_axes.scatter(
                self._colour_checker_uv[:, 0],
                self._colour_checker_uv[:, 1],
                c='white',
                marker='D',
                s=5)

        # Reference Image Axes
        self._reference_image_axes = self._figure.add_subplot(self._grid_spec[
            reference_image_indices[0], reference_image_indices[1]])
        self._reference_image_axes.set_xticks([])
        self._reference_image_axes.set_yticks([])
        self._reference_image_imshow = self._reference_image_axes.imshow(
            self.working_space_to_display_space(self._reference_image_working))

    def initialise_widgets(self):
        # *** Widgets ***
        self._disable_medidicina_CheckBox = (
            ipywidgets.widgets.Checkbox(description='Disable Medicina'))
        self._export_reference_image_Button = (
            ipywidgets.widgets.Button(description="Export Image"))
        self._save_figure_image_Button = (
            ipywidgets.widgets.Button(description="Save Figure"))

        # *** Layout ***
        self.layout.display = 'flex'
        self.layout.flex_flow = 'column'
        self.layout.align_items = 'stretch'
        self.layout.width = '{0}px'.format(self._figure_size[0] * 100 +
                                           MPL_BOX_DECORATION_WIDTH)

        self._global_controls_HBox = ipywidgets.widgets.HBox([
            self._disable_medidicina_CheckBox,
            self._export_reference_image_Button,
            self._save_figure_image_Button,
        ])

        self._controls_Tab = ipywidgets.widgets.Tab(children=[
            self._global_controls_HBox,
        ])
        self._controls_Tab.set_title(0, 'Global Controls')

        self._controls_Tab.layout = border_layout()

        self._output.layout = border_layout()
        self.children = [self._output, self._controls_Tab]

    def attach_callbacks(self):
        self._disable_medidicina_CheckBox.observe(self.toggle_medidicina)
        self._export_reference_image_Button.on_click(
            self.export_reference_image)
        self._save_figure_image_Button.on_click(self.save_figure)

    def reference_space_to_working_space(self, RGB):
        return colour.RGB_to_RGB(RGB, self._reference_space,
                                 self._working_space)

    def working_space_to_reference_space(self, RGB):
        return colour.RGB_to_RGB(RGB, self._working_space,
                                 self._reference_space)

    def working_space_to_display_space(self,
                                       RGB,
                                       apply_encoding_cctf=True,
                                       clip=True):
        RGB = colour.RGB_to_RGB(
            RGB,
            self._working_space,
            self._display_space,
            apply_encoding_cctf=apply_encoding_cctf)

        if clip:
            return np.clip(RGB, 0, 1)
        else:
            return RGB

    def update_scatter_plot(self, RGB):
        if self._enable:
            RGB = RGB[RGB[:, 1].argsort()]
            XYZ = colour.RGB_to_XYZ(RGB, self._working_space.whitepoint,
                                    self._working_space.whitepoint,
                                    self._working_space.RGB_to_XYZ_matrix)
            uv = colour.Luv_to_uv(
                colour.XYZ_to_Luv(XYZ, self._working_space.whitepoint),
                self._working_space.whitepoint)
            self._scatter_path_collection.set_offsets(uv)
            self._scatter_path_collection.set_facecolor(
                self.working_space_to_display_space(RGB))
        else:
            self._scatter_path_collection.set_offsets(self._scatter_offsets_i)
            self._scatter_path_collection.set_facecolor(
                self._scatter_facecolor_i)

    def toggle_medidicina(self, change):
        if not change:
            return

        if change['name'] == 'value':
            self._enable = not change['new']

            self.update_widget(None)

    def export_reference_image(self, change):
        path = os.path.splitext(self._reference_image_path)[0]

        colour.write_image(
            self.working_space_to_reference_space(
                self._reference_image_mapped), '{0}_{1}.exr'.format(
                    path, self.__class__.__name__))

    def save_figure(self, change):
        path = os.path.splitext(self._reference_image_path)[0]
        plt.savefig('{0}_{1}.png'.format(path, self.__class__.__name__),
                   facecolor=[0.1, 0.1, 0.1, 1], transparent=False)

    def update_widget(self, change):
        if not self._enable:
            if self._enable_chromaticity_diagram:
                self.update_scatter_plot(None)

            self._reference_image_imshow.set_data(
                self._reference_image_display)

            self._figure.canvas.draw_idle()


GamutMedicinaBaseWidget()


GamutMedicinaBaseWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0')), T…

In [6]:
GamutMedicinaBaseWidget(enable_colour_wheel=False, image_ratio=1.5)

GamutMedicinaBaseWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0')), T…

In [7]:
GamutMedicinaBaseWidget(enable_chromaticity_diagram=False, image_ratio=1.5)

GamutMedicinaBaseWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0')), T…

In [8]:
GamutMedicinaBaseWidget(
    enable_colour_wheel=False, enable_chromaticity_diagram=False, image_ratio=1)

GamutMedicinaBaseWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0')), T…

## Compression & Blending Functions

In [9]:
def tanh_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, (a + b * np.tanh((x - a) / b)), x)


def smoothstep(x, a=0, b=1):
    i = np.clip((x - a) / (b - a), 0, 1)

    return i**2 * (3 - 2 * i)


## Mansencal and Scharfenberg (2020) - Gamut Medicina HSV Control

https://community.acescentral.com/t/gamut-mapping-in-cylindrical-and-conic-spaces/2870/4

In [10]:
class GamutMedicinaHSVControlWidget(GamutMedicinaBaseWidget):
    def __init__(self,
                 reference_image=None,
                 hue_wedge=45,
                 protected_area_threshold=0.7,
                 compression_threshold=0.8,
                 RGB_to_HSV=colour.RGB_to_HSV,
                 HSV_to_RGB=colour.HSV_to_RGB,
                 *args,
                 **kwargs):

        self._hue_wedge = hue_wedge
        self._protected_area_threshold = protected_area_threshold
        self._compression_threshold = compression_threshold

        self.RGB_to_HSV = RGB_to_HSV
        self.HSV_to_RGB = HSV_to_RGB

        super().__init__(reference_image, *args, **kwargs)
        
        self._RGB_r = None
        self._H_r = self._S_r = self._V_r = None
        self._S_r_m = None

        if self._enable_colour_wheel:
            self._RGB_w = None
            self._H_w = self._S_w = self._V_w = None
            self._S_w_m = None

        if self._enable_chromaticity_diagram:
            self._RGB_d = None
            self._H_d = self._S_d = self._V_d = None
            self._S_d_m = None
        
        self._angles = None
        self._x_i = self._y_i = self._y_i = None

        if self._enable_colour_wheel:
            self._protected_area_threshold_colour_wheel_plot = None
            self._compression_threshold_colour_wheel_plot = None
            self._hue_controls_plot = None
        
        if self._enable_chromaticity_diagram:
            self._protected_area_threshold_chromaticity_diagram_plot = None
            self._compression_threshold_chromaticity_diagram_plot = None

        self._reset_protected_area_controls_Button = None
        self._protected_area_threshold_FloatSlider = None
        self._reset_compression_controls_Button = None
        self._compression_threshold_FloatSlider = None
        self._reset_hue_controls_Button = None
        self._wedge_controls = None
        self._protected_area_controls_HBox = None
        self._saturation_controls_HBox = None
        self._hue_controls_HBox = None

        self.initialize_data()
        self.initialize_axes()
        self.initialise_widgets()
        self.attach_callbacks()

        self.update_widget(None)

    def initialize_data(self):
        super().initialize_data()

        # *** Colour Wheel ***
        if self._enable_colour_wheel:
            self._RGB_w = self._colour_wheel
            self._H_w, self._S_w, self._V_w = colour.utilities.tsplit(
                self.RGB_to_HSV(self._colour_wheel))
            self._S_w_m = smoothstep(self._S_w, self._protected_area_threshold,
                                     self._compression_threshold)

        # *** Decimated Image, i.e. Scatter ***
        if self._enable_chromaticity_diagram:
            self._RGB_d = self._decimated_image_working
            self._H_d, self._S_d, self._V_d = colour.utilities.tsplit(
                self.RGB_to_HSV(self._decimated_image_working))
            self._S_d_m = smoothstep(self._S_d, self._protected_area_threshold,
                                     self._compression_threshold)

        # *** Reference Image ***
        self._RGB_r = self._reference_image_working
        self._H_r, self._S_r, self._V_r = colour.utilities.tsplit(
            self.RGB_to_HSV(self._reference_image_working))
        self._S_r_m = smoothstep(self._S_r, self._protected_area_threshold,
                                 self._compression_threshold)

        # *** Angles ***
        self._angles = np.arange(0, 360, self._hue_wedge) / 360

        # *** Initial State ***
        self._x_i = np.hstack([self._angles, 1])
        self._y_i = self._angles
        self._y_i = np.hstack([self._y_i, self._y_i[0] + 1])

    def initialize_axes(self):
        super().initialize_axes()

        circumference = np.linspace(0, np.pi * 2,
                                    self._colour_wheel_samples)
        # Colour Wheel Axes
        if self._enable_colour_wheel:
            self._protected_area_threshold_colour_wheel_plot = (
                self._colour_wheel_polar_axes.plot(
                    circumference,
                    np.full(self._colour_wheel_samples,
                            self._protected_area_threshold),
                    linestyle='dotted',
                    color='yellow')[0])
            self._compression_threshold_colour_wheel_plot = (
                self._colour_wheel_polar_axes.plot(
                    circumference,
                    np.full(self._colour_wheel_samples,
                            self._compression_threshold),
                    linestyle='dashdot',
                    color='cyan')[0])

            self._hue_controls_plot = (self._colour_wheel_polar_axes.plot(
                self._y_i * np.pi * 2,
                np.ones(self._y_i.shape),
                'o-',
                color='white')[0])

        # Chromaticity Diagram Axes
        if self._enable_chromaticity_diagram:
            self._protected_area_threshold_chromaticity_diagram_plot = (
                self._chromaticity_diagram_axes.plot(
                    circumference,
                    circumference,
                    linestyle='dotted',
                    color='yellow')[0])
            self._compression_threshold_chromaticity_diagram_plot = (
                self._chromaticity_diagram_axes.plot(
                    circumference,
                    circumference,
                    linestyle='dashdot',
                    color='cyan')[0])

    def initialise_widgets(self):
        super().initialise_widgets()

        # *** Widgets ***
        self._reset_protected_area_controls_Button = (
            ipywidgets.widgets.Button(description="Reset Protected Area"))
        self._protected_area_threshold_FloatSlider = (
            ipywidgets.widgets.FloatSlider(
                min=0.0,
                max=1.0,
                step=0.01,
                value=self._protected_area_threshold,
                description='Protected Area Threshold'))

        self._reset_compression_controls_Button = (
            ipywidgets.widgets.Button(description="Reset Saturation Controls"))
        self._compression_threshold_FloatSlider = (
            ipywidgets.widgets.FloatSlider(
                min=0.0,
                max=1.0,
                step=0.01,
                value=self._compression_threshold,
                description='Compression Threshold'))

        self._reset_hue_controls_Button = ipywidgets.widgets.Button(
            description="Reset Hue Controls")
        self._wedge_controls = [
            ipywidgets.widgets.FloatSlider(
                min=-1,
                max=1,
                step=0.01,
                value=0,
                description='{0}°'.format(int(angle * 360)))
            for angle in self._angles
        ]

        # *** Layout ***
        self._protected_area_controls_HBox = ipywidgets.widgets.HBox([
            self._reset_protected_area_controls_Button,
            self._protected_area_threshold_FloatSlider,
        ])

        self._saturation_controls_HBox = ipywidgets.widgets.HBox([
            self._reset_compression_controls_Button,
            self._compression_threshold_FloatSlider
        ])

        wedge_controls_batches = batch(self._wedge_controls, 3)
        self._hue_controls_HBox = ipywidgets.widgets.HBox(
            [ipywidgets.widgets.VBox([self._reset_hue_controls_Button])] + [
                ipywidgets.widgets.VBox(wedge_controls)
                for wedge_controls in wedge_controls_batches
            ])

        self._controls_Tab.children += (
            self._protected_area_controls_HBox,
            self._saturation_controls_HBox,
            self._hue_controls_HBox,
        )

        self._controls_Tab.set_title(1, 'Protected Area Controls')
        self._controls_Tab.set_title(2, 'Saturation Controls')
        self._controls_Tab.set_title(3, 'Hue Controls')

    def attach_callbacks(self):
        super().attach_callbacks()

        self._reset_protected_area_controls_Button.on_click(
            self.reset_protected_area_controls)
        self._protected_area_threshold_FloatSlider.observe(
            self.update_protected_area_mask, 'value')

        self._reset_compression_controls_Button.on_click(
            self.reset_compression_controls)
        self._compression_threshold_FloatSlider.observe(
            self.update_widget, 'value')

        self._reset_hue_controls_Button.on_click(self.reset_hue_controls)
        for slider in self._wedge_controls:
            slider.observe(self.update_widget, 'value')

    def update_region_colour_wheel_plot(self, V_r, region_colour_wheel_plot):
        if self._enable_colour_wheel:
            region_colour_wheel_plot.set_ydata(
                np.full(self._colour_wheel_samples, V_r))

    def update_protected_area_threshold_colour_wheel_plot(self, U_r):
        if self._enable_colour_wheel:
            self.update_region_colour_wheel_plot(
                U_r, self._protected_area_threshold_colour_wheel_plot)

    def update_compression_threshold_colour_wheel_plot(self, S_c):
        if self._enable_colour_wheel:
            self.update_region_colour_wheel_plot(
                S_c, self._compression_threshold_colour_wheel_plot)

    def update_region_chromaticity_diagram_plot(
            self, V_r, region_chromaticity_diagram_plot):
        if self._enable_chromaticity_diagram:
            HSV = colour.utilities.tstack([
                np.linspace(0, 1, self._colour_wheel_samples),
                np.full(self._colour_wheel_samples, V_r),
                np.ones(self._colour_wheel_samples)
            ])
            uv = colour.Luv_to_uv(
                colour.XYZ_to_Luv(
                    colour.RGB_to_XYZ(
                        self.HSV_to_RGB(HSV), self._working_space.whitepoint,
                        self._working_space.whitepoint,
                        self._working_space.RGB_to_XYZ_matrix),
                    self._working_space.whitepoint))

            region_chromaticity_diagram_plot.set_data(uv[:, 0], uv[:, 1])

    def update_protected_area_threshold_chromaticity_diagram_plot(self, U_r):
        if self._enable_chromaticity_diagram:
            self.update_region_chromaticity_diagram_plot(
                U_r, self._protected_area_threshold_chromaticity_diagram_plot)

    def update_compression_threshold_chromaticity_diagram_plot(self, S_c):
        if self._enable_chromaticity_diagram:
            self.update_region_chromaticity_diagram_plot(
                S_c, self._compression_threshold_chromaticity_diagram_plot)

    def update_protected_area_mask(self, change):
        if not change:
            return

        if change['name'] == 'value':
            U_r = self._protected_area_threshold_FloatSlider.value
            S_c = self._compression_threshold_FloatSlider.value

            if self._enable_colour_wheel:
                self._S_w_m = smoothstep(self._S_w, U_r, S_c)

            if self._enable_chromaticity_diagram:
                self._S_r_m = smoothstep(self._S_r, U_r, S_c)

            self._S_d_m = smoothstep(self._S_d, U_r, S_c)

        self.update_widget(None)

    def wedge_control_to_hue_offset(self, value):
        slider = self._wedge_controls[0]

        return colour.utilities.linear_conversion(
            value, [slider.min, slider.max],
            [-self._hue_wedge / 360, self._hue_wedge / 360])

    def reset_protected_area_controls(self, change):
        self._protected_area_threshold_FloatSlider.value = (
            self._protected_area_threshold)

        self.update_protected_area_mask(None)

    def reset_compression_controls(self, change):
        self._compression_threshold_FloatSlider.value = (
            self._compression_threshold)

        self.update_widget(None)

    def reset_hue_controls(self, change):
        for slider in self._wedge_controls:
            slider.value = 0

        self.update_widget(None)

    def medicina(self, RGB, H, S, V, H_x, H_y, S_c, S_m):
        S_m = S_m[..., np.newaxis]

        interpolator = scipy.interpolate.interp1d(H_x, H_y)

        HSV_c = colour.utilities.tstack(
            [interpolator(H) % 1,
             tanh_compression_function(S, S_c, 1 - S_c), V])

        return self.HSV_to_RGB(HSV_c) * S_m + RGB * (1 - S_m)

    def update_colour_wheel_axes(self, U_r, S_c, H_y, H_o):
        if self._enable_colour_wheel:
            R_w, G_w, B_w = colour.utilities.tsplit(
                self.medicina(self._RGB_w, self._H_w, self._S_w, self._V_w,
                              self._x_i, H_y, S_c, self._S_w_m))
            self._colour_wheel_imshow.set_data(
                np.clip(colour.tstack([R_w, G_w, B_w, self._A_w]), 0, 1))
            self.update_protected_area_threshold_colour_wheel_plot(U_r)
            self.update_compression_threshold_colour_wheel_plot(S_c)

            H_x = H_o + self._angles
            H_x = np.hstack([H_x, H_x[0]])
            self._hue_controls_plot.set_xdata(H_x * np.pi * 2)

    def update_chromaticity_diagram_axes(self, U_r, S_c, H_y):
        if self._enable_chromaticity_diagram:
            RGB_r_s = self.medicina(self._RGB_d, self._H_d, self._S_d,
                                    self._V_d, self._x_i, H_y, S_c,
                                    self._S_d_m).reshape(-1, 3)
            self.update_scatter_plot(RGB_r_s)
            self.update_protected_area_threshold_chromaticity_diagram_plot(U_r)
            self.update_compression_threshold_chromaticity_diagram_plot(S_c)

    def update_reference_image_axes(self, S_c, H_y):
        RGB_r = self.medicina(self._RGB_r, self._H_r, self._S_r, self._V_r,
                              self._x_i, H_y, S_c, self._S_r_m)
        self._reference_image_mapped = RGB_r
        self._reference_image_imshow.set_data(
            self.working_space_to_display_space(RGB_r))

    def update_widget(self, change):
        super().update_widget(change)

        if self._enable:
            U_r = self._protected_area_threshold_FloatSlider.value
            S_c = self._compression_threshold_FloatSlider.value

            hue_offsets = np.array([
                self.wedge_control_to_hue_offset(slider.value)
                for slider in self._wedge_controls
            ])

            H_y = -hue_offsets + self._angles
            H_y = np.hstack([H_y, H_y[0] + 1])

            if self._enable_colour_wheel:
                self.update_colour_wheel_axes(U_r, S_c, H_y, hue_offsets)

            if self._enable_chromaticity_diagram:
                self.update_chromaticity_diagram_axes(U_r, S_c, H_y)

            self.update_reference_image_axes(S_c, H_y)

            self._figure.canvas.draw_idle()


GamutMedicinaHSVControlWidget(
    '../resources/images/A009C002_190210_R0EI_Alexa_LogCWideGamut.exr',
    image_ratio=2.5)


GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [11]:
GamutMedicinaHSVControlWidget('../resources/images/A002_C029_06025K.exr')

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

## Smith (2020) - Gamut Medicina - RGB Saturation

https://community.acescentral.com/t/rgb-saturation-gamut-mapping-approach-and-a-comp-vfx-perspective/

In [12]:

class GamutMedicinaRGBSaturationWidget(GamutMedicinaBaseWidget):
    def __init__(self,
                 reference_image=None,
                 compression_threshold=0.8,
                 *args,
                 **kwargs):
        self._compression_threshold = compression_threshold

        kwargs['enable_colour_wheel'] = False

        super().__init__(reference_image, *args, **kwargs)

        self._RGB_w = None
        self._RGB_r = None
        self._RGB_d = None

        self._compression_threshold_chromaticity_diagram_plot = None

        self._reset_compression_controls_Button = None
        self._compression_threshold_FloatSlider = None

        self.initialize_data()
        self.initialize_axes()
        self.initialise_widgets()
        self.attach_callbacks()

        self.update_widget(None)

    def initialize_data(self):
        super().initialize_data()

        # *** Reference Image ***
        self._RGB_r = self._reference_image_working

        # *** Decimated Image, i.e. Scatter ***
        self._RGB_d = self._decimated_image_working

    def initialize_axes(self):
        super().initialize_axes()


    def initialise_widgets(self):
        super().initialise_widgets()

        # *** Widgets ***
        self._reset_compression_controls_Button = (
            ipywidgets.widgets.Button(
                description="Reset Compression Controls"))
        self._compression_threshold_FloatSlider = (
            ipywidgets.widgets.FloatSlider(
                min=0.0,
                max=1.0,
                step=0.01,
                value=self._compression_threshold,
                description='Compression Threshold'))

        # *** Layout ***
        self._compression_controls_HBox = ipywidgets.widgets.HBox([
            self._reset_compression_controls_Button,
            self._compression_threshold_FloatSlider
        ])

        self._controls_Tab.children += (
            self._compression_controls_HBox,
        )

        self._controls_Tab.set_title(1, 'Compression Controls')

    def attach_callbacks(self):
        super().attach_callbacks()

        self._reset_compression_controls_Button.on_click(
            self.reset_compression_controls)
        self._compression_threshold_FloatSlider.observe(
            self.update_widget, 'value')

    def reset_compression_controls(self, change):
        self._compression_threshold_FloatSlider.value = (
            self._compression_threshold)

        self.update_widget(None)

    def medicina(self, RGB, C_t=0.8):
        C_t = 1 - C_t

        L = np.max(RGB, axis=-1)[..., np.newaxis]

        D = np.abs(RGB - L) / L

        D_c = tanh_compression_function(D, C_t, 1 - C_t)

        RGB_c = L - D_c * L

        return RGB_c

    def update_chromaticity_diagram_axes(self, C_t):
        RGB_r_s = self.medicina(self._RGB_d, C_t).reshape(-1, 3)
        self.update_scatter_plot(RGB_r_s)

    def update_reference_image_axes(self, C_t):
        RGB_r = self.medicina(self._RGB_r, C_t)
        self._reference_image_mapped = RGB_r
        self._reference_image_imshow.set_data(
            self.working_space_to_display_space(RGB_r))

    def update_widget(self, change):
        super().update_widget(change)

        if self._enable:
            C_t = 1 - self._compression_threshold_FloatSlider.value

            self.update_chromaticity_diagram_axes(C_t)
            self.update_reference_image_axes(C_t)

            self._figure.canvas.draw_idle()


GamutMedicinaRGBSaturationWidget(
    '../resources/images/A009C002_190210_R0EI_Alexa_LogCWideGamut.exr',
    image_ratio=1.5)

GamutMedicinaRGBSaturationWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding…

In [13]:
GamutMedicinaHSVControlWidget(
    '../resources/images/A009C002_190210_R0EI_Alexa_LogCWideGamut.exr',
    enable_colour_wheel=False,
    image_ratio=1.5)

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [14]:
GamutMedicinaRGBSaturationWidget(
    '../resources/images/Carol Payne - J001_C001_08178N_001.exr',
    image_ratio=1.5)

GamutMedicinaRGBSaturationWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding…

In [15]:
GamutMedicinaHSVControlWidget(
    '../resources/images/Carol Payne - J001_C001_08178N_001.exr',
    enable_colour_wheel=False,
    image_ratio=1.5)

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…